In [9]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import nltk
import string
pd.options.display.float_format = '{:,}'.format

In [1]:
DIR = './'
DIR_GENRE = './genre/'

In [8]:
df1

NameError: name 'df1' is not defined

In [9]:
def count_spoilers(file_name):
    print('counting file:', file_name)
    n_review, n_sentence, n_spoiler_review, n_spoiler_sentence = 0, 0, 0, 0
    book_set, user_set = set(), set()
    print('current line: ', end='')
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            if n_review % 1000000 == 0:
                print(n_review, end=',')
            n_review += 1
            for _t, _ in d['review_sentences']:
                n_sentence += 1
                n_spoiler_sentence += _t
            n_spoiler_review += int(d['has_spoiler'])
            book_set.add(d['book_id'])
            user_set.add(d['user_id'])
    print('complete')
    print('done!')
    return n_review, n_sentence, n_spoiler_review, n_spoiler_sentence, len(book_set), len(user_set)

In [10]:
res = count_spoilers(os.path.join(DIR, 'books/goodreads_reviews_spoiler.json.gz'))
df_stats_spoiler = pd.DataFrame(res, columns=['count'], dtype=float, 
                               index=['# review', '# sentence', '# spoiler review', '# spoiler sentence',
                                      '# book', '# user'])
display(df_stats_spoiler)

counting file: ./books/goodreads_reviews_spoiler.json.gz
current line: 0,1000000,complete
done!


,count
# review,"1,378,033.0"
# sentence,"17,672,655.0"
# spoiler review,"89,627.0"
# spoiler sentence,"569,724.0"
# book,"25,475.0"
# user,"18,892.0"


#        
#             
# Code From Here
#        
#        

In [2]:
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [24]:
def load_data(file_name, head = 100000):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
#             print(count)
            data.append(d)
            
            # break if reaches the 100th line
            if (head is not None) and (count > head):
                break
    return data

In [9]:
# reviews = load_data(os.path.join(DIR, 'books/books_5.json.gz'))
len(reviews)

2000001

In [10]:
df1

NameError: name 'df1' is not defined

In [6]:
reviews[2000000]

{'overall': 4.0,
 'vote': '2',
 'verified': False,
 'reviewTime': '07 31, 2003',
 'reviewerID': 'A11U25L0QSIA5E',
 'asin': '0140106995',
 'style': {'Format:': ' Paperback'},
 'reviewerName': 'Charlie Watanabe',
 'reviewText': "I'm a sucker for books that chronicle a man's life.  I found it very enjoyable as J.J. Todd's life moves in parallel to his obsession, Roussou's CONFESSIONS.\nThis book reminded me of Mark Helprin's SOLDIER OF THE GREAT WAR.",
 'summary': "An Old Man's Story",
 'unixReviewTime': 1059609600}

In [13]:
int(len(reviews)/4)

500000

In [16]:
# label = []
review = []
summary = []
# asin_r = []
for i in range(500000,len(reviews)):
#     asin_r.append(reviews[i]['asin'])
#     label.append(reviews[i]['overall'])
    try:
        review.append(reviews[i]['reviewText'])
    except:
        review.append("")
    try:
        summary.append(reviews[i]['summary'])
    except:
        summary.append("")

In [18]:
df2 = pd.DataFrame(list(zip(review, summary)), columns =['review', 'summary'])
# df1.set_index('asin',inplace=True)
# df1["title"] = ""
df1 = pd.concat([df1,df2])
df1

,review,summary
0,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from
1,The kids loved it!,Five Stars
2,My students (3 & 4 year olds) loved this book!...,Five Stars
3,LOVE IT,Five Stars
4,Great!,Five Stars
...,...,...
1499996,"I'm a great admirer of Willian Boyd, a person ...",a very different opinion
1499997,A wonderful rampage through the twentieth cent...,Shades of Tristram Shandy (Stern) and Tom Jone...
1499998,William Boyd is a terrific storyteller. His p...,another sweeping saga by Boyd fully entertains..
1499999,This fictional memoir displays Boyd's consumma...,An Outstanding Fictional Memoir


In [28]:
df1.to_csv("books/twenty_lakh.csv")

In [12]:
df1

NameError: name 'df1' is not defined

In [247]:
below_3 = df1[df1['label']<3].copy
equal_3 = df1[df1['label']==3].copy
above_3 = df1[df1['label']>3].copy

((80982, 4), (95333, 4), (823686, 4))

In [25]:
meta = load_data(os.path.join(DIR, 'books/meta_Books.json.gz'))
len(meta)

100001

In [32]:
meta[0]['category']

[]

In [38]:
genre_set = set()
for i in range(len(meta)):
#     print(meta[i]['category'])
    for cat in meta[i]['category']:
        genre_set.add(cat)
genre_set

{'Abuse',
 'Accounting',
 'Action & Adventure',
 'Action &amp; Adventure',
 'Activities, Crafts & Games',
 'Activities, Crafts &amp; Games',
 'Addiction & Recovery',
 'Addiction &amp; Recovery',
 'Administration & Medicine Economics',
 'Administration &amp; Medicine Economics',
 'Administrative Law',
 'Adoption',
 'Africa',
 'Aging',
 'Aging Parents',
 'Agricultural Sciences',
 'Allied Health Professions',
 'Almanacs & Yearbooks',
 'Almanacs &amp; Yearbooks',
 'Alternative Medicine',
 'Americas',
 'Ancient & Medieval Literature',
 'Ancient &amp; Medieval Literature',
 'Ancient Civilizations',
 'Anger Management',
 'Animals',
 'Anthologies',
 'Anthropology',
 'Antiques & Collectibles',
 'Antiques &amp; Collectibles',
 'Anxieties & Phobias',
 'Anxieties &amp; Phobias',
 'Archaeology',
 'Architecture',
 'Art, Music & Photography',
 'Art, Music &amp; Photography',
 'Arts',
 'Arts & Literature',
 'Arts & Photography',
 'Arts &amp; Literature',
 'Arts &amp; Photography',
 'Arts, Music & Phot

In [249]:
asin_no = []
title = []
for i in range(len(meta)):
    asin_no.append(meta[i]['asin'])
    title.append(meta[i]['title'])

In [250]:
df = pd.DataFrame(list(zip(asin_no, title)), 
               columns =['asin', 'title'])
df.set_index('asin',inplace=True)
df

,title
asin,
0000092878,Biology Gods Living Creation Third Edition 10 ...
000047715X,Mksap 16 Audio Companion: Medical Knowledge Se...
0000004545,"Flex! Discography of North American Punk, Hard..."
0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal
0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple...
...,...
0888646445,"You Haven&rsquo;t Changed a Bit, and Other Sto..."
0888644817,Deep Alberta: Fossil Facts and Dinosaur Digs
0888550081,Canadian Military Handguns 1855-1985


In [251]:
for x in set(df1.index):
    try:
        df1.loc[df1.index == x, "title"] = df.loc[x]['title']
    except:
        df1.loc[df1.index == x, "title"] = ""

In [252]:
df1

,review,summary,label,title
asin,,,,
0001713353,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,5.0,"King, the Mice and the Cheese (Beginner Books)"
0001713353,The kids loved it!,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
0001713353,My students (3 & 4 year olds) loved this book!...,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
0001713353,LOVE IT,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
0001713353,Great!,Five Stars,5.0,"King, the Mice and the Cheese (Beginner Books)"
...,...,...,...,...
0061709565,"Things I liked:\n\n-Nice, easy, short read. Th...",Good If You're Looking For A Shorter Book,3.0,"Full Moon (Dark Guardian, Book 2)"
0061709565,Full Moon is the second book in the Dark Guar...,Worth ten bucks.,4.0,"Full Moon (Dark Guardian, Book 2)"
006170007X,Ransom my Heart is the first Meg Cabot book I ...,Sexy Medieval Romance (B+ Grade),4.0,Ransom My Heart


In [265]:
below_train = df1[df1['label']<3].copy()[:60000]
below_valid = df1[df1['label']<3].copy()[60000:70000]
below_test = df1[df1['label']<3].copy()[70000:80982]
equal_train = df1[df1['label']==3].copy()[:60000]
equal_valid = df1[df1['label']==3].copy()[60000:70000]
equal_test = df1[df1['label']==3].copy()[70000:80982]
above_train = df1[df1['label']>3].copy()[:60000]
above_valid = df1[df1['label']>3].copy()[60000:70000]
above_test = df1[df1['label']>3].copy()[70000:80982]

In [266]:
train = pd.concat([below_train, equal_train, above_train])
valid = pd.concat([below_valid, equal_valid, above_valid])
test = pd.concat([below_test, equal_test, above_test])

In [267]:
train.shape, valid.shape, test.shape

((180000, 4), (30000, 4), (32946, 4))

In [268]:
train.to_csv("books/train.csv")
valid.to_csv("books/valid.csv")
test.to_csv("books/test.csv")

In [162]:
df1.label.unique()

array([5., 3., 2., 4., 1.])

In [16]:
train.loc[train["label"]<3,'label']=0
train.loc[train["label"]>=3,'label']=1
# train.loc[train["label"]==3,'label']=1

In [17]:
valid.loc[valid["label"]<3,'label']=0
valid.loc[valid["label"]>=3,'label']=1
# valid.loc[valid["label"]==3,'label']=1

In [18]:
test.loc[test["label"]<3,'label']=0
test.loc[test["label"]>=3,'label']=1
# test.loc[test["label"]==3,'label']=1

In [13]:
train = pd.read_csv("books/train.csv")
valid = pd.read_csv("books/valid.csv")
test = pd.read_csv("books/test.csv")

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
def text_preproc(x):
    x = x.lower()
    x = ' '.join([word for word in x.split(' ') if word not in stop_words])
    x = " ".join(x.split("<br />"))
    x = " ".join(x.split("\n"))
    x = x.encode('ascii', 'ignore').decode()
    x = re.sub(r'https*\S+', ' ', x)
    x = re.sub(r'@\S+', ' ', x)
    x = re.sub(r'#\S+', ' ', x)
    x = re.sub(r'\'\w+', '', x)
    x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x = re.sub(r'\w*\d+\w*', '', x)
    x = re.sub(r'\s{2,}', ' ', x)
    return x

In [15]:
train['clean_text'] = train.review.astype(str).apply(text_preproc)
valid['clean_text'] = valid.review.astype(str).apply(text_preproc)
test['clean_text'] = test.review.astype(str).apply(text_preproc)

In [20]:
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding, Dropout, Dense, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [21]:
voc_size = 10000
onehot_repr = []
onehot_repr=onehot_repr+[one_hot(sents,voc_size)for sents in train["clean_text"]] 
print(len(onehot_repr))
onehot_reprv = []
onehot_reprv=onehot_reprv+[one_hot(sents,voc_size)for sents in valid["clean_text"]] 
print(len(onehot_reprv))
onehot_reprt = []
onehot_reprt=onehot_reprt+[one_hot(sents,voc_size)for sents in test["clean_text"]] 
print(len(onehot_reprt))

180000
30000
32946


In [22]:
sent_length=50
train_x=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
train_y = train.label
print(train_x)
print(train_x.shape, train_y.shape)

val_x=pad_sequences(onehot_reprv,padding='pre',maxlen=sent_length)
val_y = valid.label
print(val_x)
print(val_x.shape, val_y.shape)

test_x=pad_sequences(onehot_reprt,padding='pre',maxlen=sent_length)
test_y = test.label
print(test_x)
print(test_x.shape, test_y.shape)

[[   0    0    0 ... 1942 3692  457]
 [   0    0    0 ... 2940  457 5168]
 [   0    0    0 ... 1210 3692 2760]
 ...
 [   0    0    0 ... 8387 3918 3692]
 [   0    0    0 ...  573 1372 4548]
 [   0    0    0 ... 9312 4380 8760]]
(180000, 50) (180000,)
[[6014 5677 3080 ... 2583 9653 4356]
 [7198  179 8490 ... 5277 4986 7315]
 [2400 4952 7810 ... 3734 3807 5265]
 ...
 [2599 3429 2533 ... 3692  115 1002]
 [   0    0    0 ... 6009 8116 6649]
 [   0    0    0 ...    0 4846 3692]]
(30000, 50) (30000,)
[[   0    0    0 ... 3109 5382 1661]
 [9925  457 1428 ... 4929 8802 6545]
 [   0    0    0 ...  457 7031  318]
 ...
 [   0    0    0 ... 5797 1530 5382]
 [   0    0    0 ... 3091    5 8268]
 [   0    0    0 ...    0  166 3692]]
(32946, 50) (32946,)


In [23]:
model=Sequential()
model.add(Embedding(voc_size,100,input_length=sent_length))
model.compile('adam','mse')
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1, activation='sigmoid'))
# Adam Optimiser
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           1000000   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
dense (Dense)                (None, 512)               131584    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 1,366,593
Trainable params: 1,366,593
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(train_x, train_y, batch_size=256, 
          epochs=10,validation_data=(val_x,val_y))

Train on 180000 samples, validate on 30000 samples
Epoch 1/10
180000/180000 [==============================] - 142s 789us/sample - loss: 0.4712 - accuracy: 0.7722 - val_loss: 0.4644 - val_accuracy: 0.7729
Epoch 2/10
180000/180000 [==============================] - 137s 763us/sample - loss: 0.4180 - accuracy: 0.8041 - val_loss: 0.4684 - val_accuracy: 0.7712
Epoch 3/10
180000/180000 [==============================] - 134s 746us/sample - loss: 0.3899 - accuracy: 0.8214 - val_loss: 0.4889 - val_accuracy: 0.7677
Epoch 4/10
180000/180000 [==============================] - 146s 814us/sample - loss: 0.3620 - accuracy: 0.8362 - val_loss: 0.5237 - val_accuracy: 0.7580
Epoch 5/10
180000/180000 [==============================] - 139s 770us/sample - loss: 0.3332 - accuracy: 0.8516 - val_loss: 0.5507 - val_accuracy: 0.7548
Epoch 6/10
180000/180000 [==============================] - 142s 789us/sample - loss: 0.3054 - accuracy: 0.8662 - val_loss: 0.5946 - val_accuracy: 0.7464
Epoch 7/10
180000/180000 

In [26]:
model.evaluate(test_x, test_y)

32946/32946 [==============================] - 25s 765us/sample - loss: 0.8309 - accuracy: 0.7558


[0.8309454937369773, 0.7557822]

In [27]:
import tensorflow as tf
tf.keras.models.save_model(model,"model.h5")

In [31]:
onehot_reprx = []
s = "What a classy book"
s = [s]
onehot_reprx=onehot_reprx+[one_hot(sents,voc_size)for sents in s] 
testing_x=pad_sequences(onehot_reprx,padding='pre',maxlen=sent_length)
testing_x.shape
model.predict_classes(testing_x)

array([[1]])